# Token Counting, Frequency Distributions, TF-IDF scores, and Association Metrics across Documents in the Project

### INFO

__author__    = 'Lindsay Thomas'  
__copyright__ = 'copyright 2020, The WE1S Project'  
__license__   = 'MIT'  
__version__   = '2.0'  
__email__     = 'lindsaythomas@miami.edu'

This notebook provides a series of functions for counting the number of documents in a project containing a specific word or phrase; for obtaining uni-, bi-, and trigram frequency counts and relative frequencies; for obtaining tf-idf scores for specific words; and for utilizing some basic collocation metrics for determining the relationships between words in a project.

Some examples of questions that you can use this notebook to explore:
* How many documents in my project contain the phrase "first-generation"?
* What is the (raw and/or relative) frequency of the word "college" within a specific document in my project, and/or across all of the project's documents? What is the frequency of the phrase "private college"? What about "private college students"?
* What are some of the most distinctive words for given documents in my project? What are some of the most important words for understanding the content of documents in my project overall?
* How strongly associated is the word "private" with the word "college" in my project?
* What are some meaningful or important collocations (bigrams or trigrams, or 2- or 3-word phrases) in my project, and how can I determine and describe what it means for them to be "meaningful" or "important" in different ways?

You can also use this notebook to insert canonical source names into project json documents to obtain more accurate source totals; to download a csv containing the bibliographic information of all project documents containing a specific word or phrase for easier viewing in dfr-browser and/or to obtain individual filenames for documents you want to explore further in this notebook; and to download all documents in your project containing a specific word or phrase as .json or .txt documents for further exploration using tools like AntConc.

**Please note that you will only be able to use this notebook to count and analyze unigrams if you do not have full-text data in your document's `content` field. Please see the instructions in section 1 for more instructions.**

### GENERAL DIRECTIONS AND GLOSSARY

Each time you open this notebook, you must run all cells under the below "SETTINGS" section first, as these cells import required modules and define the functions the rest of the notebook uses. Following the "SETTINGS" section are a series of numbered sections, which are all optional and which can be run in any order. However, each numbered section after section 1 includes its own "Configure Code" cell or cells at the top of that section that you must run first, before running any other cells in that section.

**Glossary**
1. Canonical source name: The normalized title of a source. Because we obtain inconsistent titles from Lexis Nexis and ProQuest (including mis- and differently spelled versions of the same title), we have normalized all of our source titles. For example, we have recorded at least 6 different versions of the title of the newspaper the "Call and Post," some of which include "Call &amp Post, All-Ohio edition; Cleveland, Ohio", "Call & Post, All-Ohio edition; Clevel and, Ohio","Call & Post, All-Ohio edition; Cleveland, Ohi o", and "Call & Post, All-Ohio edition; Cleveland, Ohio." To account for this, we have linked all of these titles to the source's canonical name, which we have set as "call-and-post-all-ohio-edition-cleveland-ohio".
2. Token: A unit of analysis in natural language processing. Usually, this equates to what we would normally call a word; e.g., "cat", "dog", and "house" are all tokens. But "token" is also a more expansive term than "word," and can include units beyond words, e.g., punctuation marks, numbers, and bi- and trigrams (see below) can all be analyzed as "tokens."
3. Unigram: One token.
4. Bigram: A series of two tokens that occur sequentially in a text. The sentence "That cat is cute" is composed of 3 bigrams: "that cat", "cat is", "is cute". When analyzing bigrams in a text, a single bigram can also be considered as a token (i.e., "that cat" = 1 token).
5. Trigrams. A series of three tokens that occur sequentially in a text. The sentence "The cat in the tree is cute" is composed of 5 trigrams: "the cat in", "cat in the", "in the tree", "the tree is", "tree is cute". When analyzing trigrams in a text, a single trigram can also be considered as a token (i.e., "tree is cute" = 1 token).
6. Frequency count: A raw count of the number of times each token (or, in this notebook, each bi- or trigram) appears in a document and/or across all of the documents in the project.
7. Relative frequency: A percentage value indicating the raw frequency count of each word (or bi- or trigram) in a given document and/or across all of the documents in the project as compared to the total number of tokens (or bi- or trigrams) in that document document and/or across all of the documents in the project. This value is generally more meaningful than a raw frequency count because it accounts for the frequency of a given token relative to the other tokens in a document or a project.
8. Tf-idf: Term frequency - inverse document frequency. A metric for understanding the importance of words within a given document and across a project as a whole.
9. Collocation: A phrase consisting of two or more words that commonly occur together in a given context, e.g., "social media" or "hot coffee."

**TECHNICAL NOTE:** This notebook uses the NLTK package to build a custom tokenizer to tokenize project uni-, bi-, and trigrams, pulling raw text from each json document's `content` field. Its tokenization mechanisms are therefore different than those used to tokenize each document during topic modeling and during preprocessing (if you have implemented the WE1S preprocessor). To maximize flexibility, it was better in this notebook to build a custom tokenizer than it was to use the `features` field in our json documents, even for unigram tokenization. See the module [README.md]('README.md') for more information about this.

## Settings

In [ ]:
# Python imports
import os
import csv
from pathlib import Path
from IPython.display import display, HTML

# Define paths
current_dir     = %pwd
current_pathobj = Path(current_dir)
project_dir     = str(current_pathobj.parent.parent)
current_reldir  = current_dir.split("/write/")[1]
data_dir        = project_dir + '/project_data'
json_dir        = project_dir + '/project_data/json'
stopword_file   = '/home/jovyan/write/pub/templates/project_template/modules/topic_modeling/scripts/we1s_standard_stoplist.txt'

# Import scripts
%run scripts/count_tokens.py
%run scripts/source.py

# Display the project directory
display(HTML('<p><strong>Project Directory:</strong> ' + project_dir + '</p>'))


## 1. Count the Number of Documents in your Project Containing a Specific Word or Phrase

The cells in this section count the number of documents containing a specific word or phrase. They save the bibliographic information of these documents to a dataframe, which you can save as a csv and download to your own machine for further exploration. This section also allows you to download the documents containing this word or phrase themselves, as either json or txt files, so that you can explore them further using tools like AntConc.

#### A) Configure Code

Make sure that you have run the cell at the very top of this notebook, under "Settings", before proceeding. You must run all of the cells under this "Configure Code" section, even if you do not change the values.

The first step is to configure which field you would like to use as your input. You do this using the `content_field` variable. If your data includes full-text data in the `content` field, you can set `content_field = 'content'`. If you have data in your `bag_of_words` field, set `content_field = 'bag_of_words'`. If your data is in the `features` field, set `content_field = 'features'`. **Please note that if you use EITHER `bag_of_words` OR `features` as your text input field, you will only be able to count unigrams. See the module's [README.md]('README.md') file for more information on these fields.**

You follow the below format exactly:

`content_field = 'content'`

You must include quotation marks around your word or phrase because you are storing it as a string in the `required_phrase` variable.

In [ ]:
## you may select 'content', 'features', or 'bag_of_words'

content_field = 'bag_of_words'

What word or phrase do you want to search for? Insert it below. You may only search for unigrams, bigrams, or trigrams. The code will return all instances of that word or phrase that occur across all project documents, regardless of whether they are upper- or lower-case. However, if searching for a phrase, keep in mind that the computer will search for *that phrase exactly as you enter it* (except for letter case). If the phrase you want to search for contains a stop word, like "first generation" (without the hyphen -- the stopword here is "first"), you should set `set_stopwords` below to `False` (see the second configuration cell below). You can view the standard list of WE1S stopwords used for all of our processes in the [project template folder](http://harbor.english.ucsb.edu:11111/edit/write/templates/multiple_topics_template/modules/02_MALLET/vectors/we1s_standard_stoplist.txt). To account for hyphenated phrases and other uses of punctuation, the cells below **will not remove hyphens**.

You should enter your desired word or phrase according to the following format exactly:

`required_phrase = 'your word or phrase here'`

You must include quotation marks around your word or phrase because you are storing it as a string in the `required_phrase` variable.

In [ ]:
# set the word or phrase to search for
required_phrase = ''

Is your word or phrase a unigram, bigram, or trigram? Configure the `set_length` variable below according to what you are searching for. The default is unigrams; to count bigrams or trigrams, comment out the unigram line, and uncomment the bi- or trigram line. **Only one line below should be uncommented when you run the cell.**

**Note:** Because this code does not strip hyphens, hyphenated words like "first-generation" are considered unigrams.

In [ ]:
set_length = 'unigram'
# set_length = 'bigram'
# set_length = 'trigram'

Run the below cell to make sure you have set the `set_length` variable correctly. This cell will print the current value of the `set_length` variable. If you have set the wrong value, re-run the above cell with the correct setting.

In [ ]:
print(set_length)

The below cell strips the following punctuation from project documents. It will **NOT** strip hyphens, single or double, in order to account for hyphenated words and phrases such as "first-generation". Because this punctuation list is bespoke and not standardized (standardized options strip hyphens), some punctuation marks or other non-Unicode characters may make it through. You do not need to change anything about the below cell (unless you are interested in the frequency of punctuation marks, or @ signs, etc.), but you do need to run it. If you do not want to remove punctuation from your documents, you should set the `punctuations` variable to an empty string:

`punctuations = ""`

In [ ]:
# define punctuation to strip
punctuations = "_______________________\'m\'n\'ve!()[]{};:\'\"\,<>./?@#$%^&*_~''``''"

Do you want to delete stop words before obtaining word frequencies? The default setting is to delete stop words, but if your data has already had the stop words you want removed, you should change the value of `set_stopwords` to `False`. It is also generally recommended to delete stop words from a document before obtaining bi- and/or trigram frequencies. This will result in "inexact" bi- and trigrams, however, as any stop words will be deleted *before* tokenization into bi- or trigrams. If you are interested in specific bi- or trigrams that contain stop words (such as "first generation", without a hyphen -- the stop word is "first"), you may want to create a custom stop word list. 

You can view the standard list of WE1S stopwords used for all of our processes in the project template folder (the relative path is `project_name/modules/topic_modeling/scripts/we1s_standard_stoplist.txt`). This is the stop word list that will be applied to your data. You can edit this file for your project as much as you would like. You can also use your own custom stop word list. If you would like to use a different stop word list, format that list so it looks like the WE1S stopword list (one word per line, plain text file), upload that list to your project and configure the `stopword_file` variable in the "Settings" cell to the custom stop word file's path.

If you would like to the stop word setting, change `True` to `False` below. It should look like this exactly:

`set_stopwords = False`

Unlike above, you do not need quotation marks around `False` because it is a Boolean, not a string.

In [ ]:
# delete stopwords from content fields before obtaining word frequencies.
# if set to True, stop words will be deleted. If set to false, stop words will not be deleted.
set_stopwords = True

**If you are using WE1S data:** Would you like to use a document's canonical source name -- i.e., the `source` field -- in obtaining the titles of publication sources to display in your dataframe/csv file? The default setting is *not* to use  the `source` field. We recommend changing this setting to `True` if you are using WE1S data.

If you would like to use canonical source names as the titles that appear in your dataframe and/or csv, format the below setting like this exactly:

`source_set = True`

Again, you do not need quotation marks around `True` here as it is a Boolean.

In [ ]:
# source_set feature.
# if set to False, the source field will not be used. 
# if set to True, the source field will be used.

source_set = False

Run the below cell to set the name of the csv file you will save your results to. You do not need to change anything about the below cell.

In [ ]:
# the name of the csv file to save counts to
csv_file = 'counts_' + required_phrase + '.csv'

#### B) Run Code and Visualize Results

Run the below cell to create a dataframe displaying bibliographic information about each document in the project containing your search word or phrase and the number of times that word or phrase occurs in each document (in the "Count" column). The dataframe will print as output, sorted in descending order (from most to least) by count.

In [ ]:
# overall_count implementation
file_list, df, bad_jsons = overall_count(json_dir, content_field, required_phrase, set_stopwords, source_set, set_length, 
                                         punctuations, stopword_file)

if df != None:
    df_overall_count = df.sort_values('Count', ascending=False)


**View the dataframe for these counts.** The below cell uses a [QGrid](https://github.com/quantopian/qgrid) widget to display count results in a dataframe. Click a column label to sort by that column. Click it again to reverse sort. Click the filter icon to the right of the column label to apply filters (for instance, reducing the table to only documents from specific sources). You can re-order the columns by dragging the column label.

In [ ]:
qgrid_widget = qgrid.show_grid(df_overall_count, grid_options=grid_options, show_toolbar=False)

qgrid_widget

**Save the dataframe to a csv file.** The cell below will save the version of the dataframe you see displayed in the cell above. To save the full version of the dataframe (disregarding any filtering, etc you have done in the qgrid dataframe), skip the next cell, uncomment the code in the cell below it, and run that cell. 

Either cell will create a csv file in this module directory called "counts_YOUR-SEARCH-TERM-HERE.csv". You can download this csv for further processing using Excel, and/or to keep as a record for filename extraction (see section 2 below).

In [ ]:
# Save version of dataframe you see above to csv
changed_df = qgrid_widget.get_changed_df()

changed_df.to_csv(csv_file, index_label = 'Index')

In [ ]:
## Save original dataframe to csv, disregarding any changes you made in qgrid

# df_overall_count.to_csv(csv_file, index_label = 'Index')

Run the below cell to see the number of json documents in your project that failed to load correctly (and so were not read or considered in creating count totals). If this number is large, this will significantly affect the results you can obtain using this notebook.

In [ ]:
len(bad_jsons)

Run the below cell to see the total number of documents in this project that contain your search word or phrase. You must have run the cell that produces a dataframe for this cell to work (i.e., the cell that implements the `overall_count` function above).

In [ ]:
len(file_list)

**Download project json documents**: Would you like to download the documents containing your search word or phrase as json files? This is optional. Run the cell below to create a folder containing these json documents and zip that folder up for easy downloading. This cell will create a zip folder titled "YOUR-SEARCH-TERM-HERE_json.zip" in this module directory that you can download to your machine. Json documents will include all document metadata and content data.

In [ ]:
# Copy json files in file_list to separate folder and zip folder up for easy downloading.

json_path = current_dir + '/' + required_phrase + '_json'
if os.path.exists(json_path) == True:
    shutil.rmtree(json_path)
    os.mkdir(json_path)
else:
    os.mkdir(json_path)

for file in file_list:
    fpath = json_dir + '/' + file
    shutil.copy(fpath, json_path)

# make it a zip file
zpath_json = json_path + '.zip'
if os.path.exists(zpath_json) == True:
    shutil.rmtree(zpath_json)
    shutil.make_archive(json_path, 'zip', json_path)
else:
    shutil.make_archive(json_path, 'zip', json_path)

# remove the directory (but not the zip file)
shutil.rmtree(json_path)

**Download content field of project json documents as txt files**: Would you like to download the documents containing your search word or phrase as txt files? This is optional. Run the cell below to create a folder containing these json documents and zip that folder up for easy downloading. This cell will create a zip folder titled "YOUR-SEARCH-TERM-HERE_txt.zip" in this module directory that you can download to your machine for further exploration using tools like AntConc. Txt files will **just include the content of each project document**; they will not include any document metadata. Txt filenames will be the same as json filenames.

In [ ]:
# Copy content fields of json files in file_list to txt files in separate folder and zip folder up for easy downloading.

txt_path = current_dir + '/' + required_phrase + '_txt'

if os.path.exists(txt_path) == True:
    shutil.rmtree(txt_path)
    os.mkdir(txt_path)
else:
    os.mkdir(txt_path)
    
for file in file_list:
    fpath = json_dir + '/' + file
    slug = file.split('.')[0]
    txtpath = txt_path + '/' + slug + '.txt'
    with open(fpath) as fin:
        json_data = json.loads(fin.read())
        json_content = json_data['content']
    with open(txtpath, 'w') as fout:
        fout.write(json_content)

# make it a zip file
zpath_txt = txt_path + '.zip'
if os.path.exists(zpath_txt) == True:
    shutil.rmtree(zpath_txt)
    shutil.make_archive(txt_path, 'zip', txt_path)
else:
    shutil.make_archive(txt_path, 'zip', txt_path)

# remove the directory (but not the zip file)
shutil.rmtree(txt_path)

## 2. Obtain the Frequency of Uni-, Bi-, and/or Trigrams within a Single Project Document 

Use this section to obtain word frequency counts and distributions for uni-, bi-, and/or trigrams within a single project document. See section 3 to obtain frequencies for all project documents. You must run the **Configure code** cells at the top of this section for the code to work. If you run the below cells the first time counting unigrams, you can re-run them counting bigrams and/or trigrams if you change the configurations appropriately. If you are re-running this section, you should start at the top of the section and reconfigure the code before re-running anything.

This section requires that you provide the full file path for the specific json document you want to explore in more detail. The filepaths of documents containing the specific word or phrase you searched for in section 2 can be found in the csv of bibliographic information and counts you downloaded in that section. You can select specific documents to explore further in this section using that csv file.

#### A) Configure Code

Make sure that you have run the cell at the very top of this notebook, under "Settings", before proceeding. You must run all of the cells under this "Configure Code" section, even if you do not change the values.

Enter **the full file path** of the specific document you want to explore in more detail in the cell below. You must enter the full file path, which is printed in the csv you downloaded in section 2. You need to enter the full path as it appears on the csv you downloaded in section 2, following the below format exactly:

`fpath = 'full/filepath/here/it/should/end/in/.json'`

You need quotes around the file path because you are entering it as a string.

In [ ]:
# define file path for document you want to explore in more detail

fpath = ''

The next step is to configure which field you would like to use as your input. You do this using the `content_field` variable. If your data includes full-text data in the `content` field, you can set `content_field = 'content'`. If you have data in your `bag_of_words` field, set `content_field = 'bag_of_words'`. If your data is in the `features` field, set `content_field = 'features'`. **Please note that if you use EITHER `bag_of_words` OR `features` as your text input field, you will only be able to count unigrams. See the module's [README.md]('README.md') file for more information on these fields.**

You follow the below format exactly:

`content_field = 'content'`

You must include quotation marks around your word or phrase because you are storing it as a string in the `required_phrase` variable.

In [ ]:
## you may select 'content', 'features', or 'bag_of_words'

content_field = 'content'

Do you want to obtain frequencies of unigrams, bigrams, or trigrams? Configure the `set_length` variable below according to what you would like to count. The default is unigrams; to count bigrams or trigrams, comment out the unigram line, and uncomment the bi- or trigram line. **Only one line below should be uncommented when you run the cell.**

**Note:** Because this code does not strip hyphens (see below), hyphenated words like "first-generation" are considered unigrams.

In [ ]:
set_length = 'unigram'
# set_length = 'bigram'
# set_length = 'trigram'

Run the below cell to make sure you have set the `set_length` variable correctly. This cell will print the current value of the `set_length` variable. If you have set the wrong value, re-run the above cell with the correct setting.

In [ ]:
print(set_length)

The below cell strips the following punctuation from project documents. It will **NOT** strip hyphens, single or double, in order to account for hyphenated words and phrases such as "first-generation". Because this punctuation list is bespoke and not standardized (standardized options strip hyphens), some punctuation marks or other non-Unicode characters may make it through. You do not need to change anything about the below cell (unless you are interested in the frequency of punctuation marks, or @ signs, etc.), but you do need to run it.

In [ ]:
# define punctuation to strip
punctuations = "_______________________\'m\'n\'ve!()[]{};:\'\"\,<>./?@#$%^&*_~''``''"


Do you want to delete stop words before obtaining word frequencies? The default setting is to delete stop words, but if your data has already had the stop words you want removed, you should change the value of `set_stopwords` to `False`. It is also generally recommended to delete stop words from a document before obtaining bi- and/or trigram frequencies. This will result in "inexact" bi- and trigrams, however, as any stop words will be deleted *before* tokenization into bi- or trigrams. If you are interested in specific bi- or trigrams that contain stop words (such as "first generation", without a hyphen -- the stop word is "first"), you may want to create a custom stop word list. 

You can view the standard list of WE1S stopwords used for all of our processes in the project template folder (the relative path is `project_name/modules/topic_modeling/scripts/we1s_standard_stoplist.txt`). This is the stop word list that will be applied to your data. You can edit this file for your project as much as you would like. You can also use your own custom stop word list. If you would like to use a different stop word list, format that list so it looks like the WE1S stopword list (one word per line, plain text file), upload that list to your project and configure the `stopword_file` variable in the "Settings" cell to the custom stop word file's path.

If you would like to the stop word setting, change `True` to `False` below. It should look like this exactly:

`set_stopwords = False`

Unlike above, you do not need quotation marks around `False` because it is a Boolean, not a string.

In [ ]:
# set stop word setting 
# default is to delete stop words, which means setting set_stopwords to True.
# to keep stop words, set set_stopwords to False.
set_stopwords = True


#### B) Run Code and Visualize Results

Run the code below to obtain token frequencies and explore and visualize the results in various ways. You must run the below cell first; all other cells in this section are optional.

In [ ]:
# obtain frequencies for your selected document
finder, freq = frequency_single_file(fpath, content_field, set_stopwords, punctuations, set_length, stopword_file)

**Frequency Counts**: Produce a dataframe of the frequency counts for all tokens (uni-, bi-, or trigrams, whichever you selected above) in the document. The dataframe will be ordered by count, from the tokens with the highest frequency to those with the lowest.

In [ ]:
# produce a dataframe of counts for all unigrams in the document

df_frequencies = pd.Series(freq, name='Count')

df_frequencies.index.name = 'Token'

df_frequencies = df_frequencies.reset_index()

df_frequencies = df_frequencies.sort_values('Count', ascending=False) 

**View the dataframe for these counts.** The below cell uses a [QGrid](https://github.com/quantopian/qgrid) widget to display count results in a dataframe. Click a column label to sort by that column. Click it again to reverse sort. Click the filter icon to the right of the column label to apply filters (for instance, reducing the table to only documents from specific sources). You can re-order the columns by dragging the column label.

In [ ]:
qgrid_widget = qgrid.show_grid(df_frequencies, grid_options=grid_options, show_toolbar=False)

qgrid_widget

**Save the dataframe to a csv file.** The cell below will save the version of the dataframe you see displayed in the cell above. To save the full version of the dataframe (disregarding any filtering, etc you have done in the qgrid dataframe), skip the next cell, uncomment the code in the cell below it, and run that cell. 

Either cell will create a csv file in this module directory called "FILE-PATH-OF-JSON-DOCUMENT_counts.csv". You can download this csv for further processing using Excel, Google Sheets, etc.

In [ ]:
# create csv file name
csv_file_slug = os.path.basename(fpath).split('.')[0]
csv_file_counts = csv_file_slug + '_' + set_length + '_counts.csv'

# Save version of dataframe you see above to csv
changed_df = qgrid_widget.get_changed_df()

changed_df.to_csv(csv_file_counts, index_label = 'Index')

In [ ]:
## Save original dataframe to csv, disregarding any changes you made in qgrid

## create csv file name
# csv_file_slug = os.path.basename(fpath).split('.')[0]
# csv_file_counts = csv_file_slug + '_' + set_length + '_counts.csv'

# # save the above dataframe to a csv file
# df_frequencies.to_csv(csv_file_counts, index_label = 'Index')

Plot the top n tokens (uni-, bi-, or trigrams, whatever you selected above) with their counts. It is not recommended that you plot a graph of *all* token frequencies in a document, even with stop words removed, as it will be hard to read. Instead, define the top number you want to see (i.e., the top 30 results), and plot those. The example provided below is 30. If you would like to change this, please follow the below format exactly:

`top_n = 40`

You do not need quotes around the number, as it is an integer, not a string.

In [ ]:
# define the number of top results you want to see in your plot
top_n = 30

In [ ]:
# plot frequency counts for your given document
%matplotlib inline

#plotting top 30 words
freq.plot(top_n)

Find the frequency count of a specific token. First, check the `set_length` variable to make sure you have tokenized on the proper token length. I.e., if you are searching for a specific word, you should have tokenized on unigrams above when you ran the `frequency_single_file` function. If you ran the function using the unigram setting, but would now like to search for a bigram or trigram, you must re-run the `frequeny_single_file` function above (the first cell under "B) Run Code and Visualize Results").

In [ ]:
print(set_length)

Enter the token you want to search for. If you are searching for a unigram, just enter it below. Remember that the tokenization method is case-insensitive (i.e., it is not possible to differentiate between "book" and "Book"). If you are searching for a specific bigram or trigram, enter it with spaces separating each part. Remember that if you deleted stopwords above, your bi- or trigram should not contain stopwords. Follow the below format exactly:

`token = 'unigram'`

OR

`token = 'bigram example'`

OR

`token = 'trigram example here'`

You need to put quotes around your token because you are entering it as a string.



In [ ]:
token = ''

Run the below cell to obtain the frequency count for your selected token. If your token as you have entered it above is not in your project, the frequency count will be 0.

In [ ]:
# get frequency count
if set_length == 'unigram':
    print(token + ': ' + str(freq[token]))
if set_length == 'bigram':
    token1 = token.split()[0]
    token2 = token.split()[1]
    print(token + ': ' + str(freq[token1, token2]))
if set_length == 'trigram':
    token1 = token.split()[0]
    token2 = token.split()[1]
    token3 = token.split()[2]
    print(token + ': ' + str(freq[token1, token2, token3]))

**Relative Frequency**: The cells above gave you the raw counts of tokens in your selected document. However, raw counts can be misleading, as they do not account for the relative frequency of tokens within a document (i.e., how many times a given token occurs *relative* to all other tokens). Relative frequencies are obtained by dividing the raw frequency count for each token by the total number of tokens in a document. To do this, run the below cell. You must run the below cell to complete any other cells in this portion of the notebook. After running this cell, you can create a dataframe of relative frequencies, save that dataframe, plot the top results, and obtain the relative frequency of a given token.

In [ ]:
# obtain the total number of tokens in the document
norm = freq.N()

# make a copy of the freq object 
freq_dist = freq.copy()

# replace raw frequency counts with relative frequency values
for key in freq_dist.keys():
        freq_dist[key] = float(freq_dist[key]) / norm

Produce a dataframe of the relative frequency values for all tokens (uni-, bi-, or trigrams, whichever you selected above) in the document. The dataframe will be ordered by relaive frequency value, from the tokens with the highest relative frequency to those with the lowest. 

The values will be listed as decimals, and you should interpret them as percentage values. For example, if the top token listed is "town" with a relative frequency of .0244, then this means that the token "town" accounts for 2.44% of the total number of tokens in the document.

In [ ]:
df_relfrequencies = pd.Series(freq_dist, name='Frequency Distribution')

df_relfrequencies.index.name = 'Token'

df_relfrequencies = df_relfrequencies.reset_index()

df_relfrequencies = df_relfrequencies.sort_values('Frequency Distribution', ascending=False) 

df_relfrequencies

**View the dataframe for these counts.** The below cell uses a [QGrid](https://github.com/quantopian/qgrid) widget to display count results in a dataframe. Click a column label to sort by that column. Click it again to reverse sort. Click the filter icon to the right of the column label to apply filters (for instance, reducing the table to only documents from specific sources). You can re-order the columns by dragging the column label.

In [ ]:
qgrid_widget = qgrid.show_grid(df_relfrequencies, grid_options=grid_options, show_toolbar=False)

qgrid_widget

**Save the dataframe to a csv file.** The cell below will save the version of the dataframe you see displayed in the cell above. To save the full version of the dataframe (disregarding any filtering, etc you have done in the qgrid dataframe), skip the next cell, uncomment the code in the cell below it, and run that cell. 

Either cell will create a csv file in this module directory called "FILE-PATH-OF-JSON-DOCUMENT_relfreq.csv". You can download this csv for further processing using Excel, Google Sheets, etc.

In [ ]:
# create csv file name
csv_file_slug = os.path.basename(fpath).split('.')[0]
csv_file_counts = csv_file_slug + '_' + set_length + '_relfreq.csv'

# Save version of dataframe you see above to csv
changed_df = qgrid_widget.get_changed_df()

changed_df.to_csv(csv_file_counts, index_label = 'Index')

In [ ]:
# # create csv file name
# csv_file_slug = os.path.basename(fpath).split('.')[0]
# csv_file_counts = csv_file_slug + '_' + set_length + '_relfreq.csv'

# # save the above dataframe to a csv file
# df_relfrequencies.to_csv(csv_file_counts, index_label = 'Index')

Plot the top n tokens (uni-, bi-, or trigrams, whatever you selected above) with their relative frequencies. It is not recommended that you plot a graph of *all* token frequencies in a document, even with stop words removed, as it will be hard to read. Instead, define the top number you want to see (i.e., the top 30 results), and plot those. The example provided below is 30. If you would like to change this, please follow the below format exactly:

`top_n = 40`

You do not need quotes around the number, as it is an integer, not a string.

In [ ]:
# define the number of top results you want to see in your plot
top_n = 30

In [ ]:
# plot frequency counts for your given document
%matplotlib inline

#plotting top 30 words
freq_dist.plot(top_n, title = 'Relative Frequencies')

Find the relative frequency of a specific token. First, check the `set_length` variable to make sure you have tokenized on the proper token length. I.e., if you are searching for a specific word, you should have tokenized on unigrams above when you ran the `frequency_single_file` function. If you ran the function using the unigram setting, but would now like to search for a bigram or trigram, you must re-run the `frequeny_single_file` function above (the first cell under "B) Run Code and Visualize Results").

In [ ]:
print(set_length)

Enter the token you want to search for. If you are searching for a unigram, just enter it below. Remember that the tokenization method is case-insensitive (i.e., it is not possible to differentiate between "book" and "Book"). If you are searching for a specific bigram or trigram, enter it with spaces separating each part. Remember that if you deleted stopwords above, your bi- or trigram should not contain stopwords. Follow the below format exactly:

`token = 'unigram'`

OR

`token = 'bigram example'`

OR

`token = 'trigram example here'`

You need to put quotes around your token because you are entering it as a string.



In [ ]:
token = ''

Run the below cell to obtain the relative frequency for your selected token. If your token as you have entered it above is not in the corpus, the relative frequency will be 0.

In [ ]:
# get relative frequency count
if set_length == 'unigram':
        print(token + ': ' + str(freq_dist[token]))
if set_length == 'bigram':
    token1 = token.split()[0]
    token2 = token.split()[1]
    print(token + ': ' + str(freq_dist[token1, token2]))
if set_length == 'trigram':
    token1 = token.split()[0]
    token2 = token.split()[1]
    token3 = token.split()[2]
    print(token + ': ' + str(freq_dist[token1, token2, token3]))

## 3. Obtain the Frequency of Uni-, Bi-, and/or Trigrams across the Project as a Whole

Use this section to obtain word frequency counts and distributions for uni-, bi-, and/or trigrams across all of the project's documents. See section 2 to obtain frequencies for a specific json document within the project.

You must run the **Configure code** cells at the top of this section for the code to work. If you run the below cells the first time counting unigrams, you can re-run them counting bigrams and/or trigrams if you change the settings appropriately (see below). If you are re-running this section, you should start here and reconfigure the code before re-running anything.

#### A) Configure Code

Make sure that you have run the cell at the very top of this notebook, under "Settings", before proceeding. You must run all of the cells under this "Configure Code" section, even if you do not change the values.

The first step is to configure which field you would like to use as your input. You do this using the `content_field` variable. If your data includes full-text data in the `content` field, you can set `content_field = 'content'`. If you have data in your `bag_of_words` field, set `content_field = 'bag_of_words'`. If your data is in the `features` field, set `content_field = 'features'`. **Please note that if you use EITHER `bag_of_words` OR `features` as your text input field, you will only be able to count unigrams.**

You follow the below format exactly:

`content_field = 'content'`

You must include quotation marks around your word or phrase because you are storing it as a string in the `required_phrase` variable.

In [ ]:
## you may select 'content', 'features', or 'bag_of_words'

content_field = 'content'

Do you want to obtain frequencies of unigrams, bigrams, or trigrams? Configure the `set_length` variable below according to what you would like to count. The default is unigrams; to count bigrams or trigrams, comment out the unigram line, and uncomment the bi- or trigram line. **Only one line below should be uncommented when you run the cell.**

**Note:** Because this code does not strip hyphens (see below), hyphenated words like "first-generation" are considered unigrams.

In [ ]:
set_length = 'unigram'
# set_length = 'bigram'
# set_length = 'trigram'

Run the below cell to make sure you have set the `set_length` variable correctly. This cell will print the current value of the `set_length` variable. If you have set the wrong value, re-run the above cell with the correct setting.

In [ ]:
print(set_length)

The below cell strips the following punctuation from project documents. It will **NOT** strip hyphens, single or double, in order to account for hyphenated words and phrases such as "first-generation". Because this punctuation list is bespoke and not standardized (standardized options strip hyphens), some punctuation marks or other non-Unicode characters may make it through. You do not need to change anything about the below cell (unless you are interested in the frequency of punctuation marks, or @ signs, etc.), but you do need to run it.

In [ ]:
# define punctuation to strip
punctuations = "_______________________\'m\'n\'ve!()[]{};:\'\"\,<>./?@#$%^&*_~''``''"


Do you want to delete stop words before obtaining word frequencies? The default setting is to delete stop words, but if your data has already had the stop words you want removed, you should change the value of `set_stopwords` to `False`. It is also generally recommended to delete stop words from a document before obtaining bi- and/or trigram frequencies. This will result in "inexact" bi- and trigrams, however, as any stop words will be deleted *before* tokenization into bi- or trigrams. If you are interested in specific bi- or trigrams that contain stop words (such as "first generation", without a hyphen -- the stop word is "first"), you may want to create a custom stop word list. 

You can view the standard list of WE1S stopwords used for all of our processes in the project template folder (the relative path is `project_name/modules/topic_modeling/scripts/we1s_standard_stoplist.txt`). This is the stop word list that will be applied to your data. You can edit this file for your project as much as you would like. You can also use your own custom stop word list. If you would like to use a different stop word list, format that list so it looks like the WE1S stopword list (one word per line, plain text file), upload that list to your project and configure the `stopword_file` variable in the "Settings" cell to the custom stop word file's path.

If you would like to the stop word setting, change `True` to `False` below. It should look like this exactly:

`set_stopwords = False`

Unlike above, you do not need quotation marks around `False` because it is a Boolean, not a string.

In [ ]:
# set stop word setting 
# default is to delete stop words, which means setting set_stopwords to True.
# to keep stop words, set set_stopwords to False.
set_stopwords = True


#### B) Run Code and Visualize Results

Run the code below to obtain token frequencies and explore and visualize the results in various ways. You must run the below cell first; all other cells in this section are optional.

In [ ]:
# obtain frequencies for your project as a whole
all_finders_freq, all_finders_list, freq, bad_jsons = frequency_dir(json_dir, content_field, set_stopwords, 
                                                                    punctuations, set_length, stopword_file)


Run the below cell to see the number of json documents in your project that failed to load correctly (and so were not read or considered in creating count totals). If this number is large, this will significantly affect the results you can obtain using this notebook; notify Lindsay if this is the case.

In [ ]:
len(bad_jsons)

**Frequency Counts**: Produce a dataframe of the frequency counts for all tokens (uni-, bi-, or trigrams, whichever you selected above) in the project. The dataframe will be ordered by count, from the tokens with the highest frequency to those with the lowest.

In [ ]:
# produce a dataframe of counts for all tokens in the project

df_frequencies = pd.Series(freq, name='Count')

df_frequencies.index.name = 'Token'

df_frequencies = df_frequencies.reset_index()

df_frequencies = df_frequencies.sort_values('Count', ascending=False) 

**View the dataframe for these counts.** The below cell uses a [QGrid](https://github.com/quantopian/qgrid) widget to display count results in a dataframe. Click a column label to sort by that column. Click it again to reverse sort. Click the filter icon to the right of the column label to apply filters (for instance, reducing the table to only documents from specific sources). You can re-order the columns by dragging the column label.

In [ ]:
qgrid_widget = qgrid.show_grid(df_frequencies, grid_options=grid_options, show_toolbar=False)

qgrid_widget

**Save the dataframe to a csv file.** The cell below will save the version of the dataframe you see displayed in the cell above. To save the full version of the dataframe (disregarding any filtering, etc you have done in the qgrid dataframe), skip the next cell, uncomment the code in the cell below it, and run that cell. 

Either cell will create a csv file in this module directory called "PROJECT-NAME_LENGTH_counts.csv". You can download this csv for further processing using Excel, Google Sheets, etc.

In [ ]:
# create csv file name
csv_file_counts = project_dir + '_' + set_length + '_counts.csv'

# Save version of dataframe you see above to csv
changed_df = qgrid_widget.get_changed_df()

changed_df.to_csv(csv_file_counts, index_label = 'Index')

In [ ]:
# # create csv file name
# csv_file_counts = project_dir + '_' + set_length + '_counts.csv'

# # save the above dataframe to a csv file
# df_frequencies.to_csv(csv_file_counts, index_label = 'Index')

Plot the top n tokens (uni-, bi-, or trigrams, whatever you selected above) with their counts. It is not recommended that you plot a graph of *all* token frequencies in a document, even with stop words removed, as it will be hard to read. Instead, define the top number you want to see (i.e., the top 30 results), and plot those. The example provided below is 30. If you would like to change this, please follow the below format exactly:

`top_n = 40`

You do not need quotes around the number, as it is an integer, not a string.

In [ ]:
# define the number of top results you want to see in your plot
top_n = 30

In [ ]:
# plot frequency counts for your given document
%matplotlib inline

#plotting top 30 words
freq.plot(top_n)

Find the frequency count of a specific token across all documents in your project. See section 5 to find the tf-idf values of a specific token across all documents in your project. First, check the `set_length` variable to make sure you have tokenized on the proper token length. I.e., if you are searching for a specific word, you should have tokenized on unigrams above when you ran the `frequency_dir` function. If you ran the function using the unigram setting, but would now like to search for a bigram or trigram, you must re-run the `frequeny_dir` function above (the first cell under "B) Run Code and Visualize Results").

In [ ]:
print(set_length)

Enter the token you want to search for. If you are searching for a unigram, just enter it below. Remember that the tokenization method is case-insensitive (i.e., it is not possible to differentiate between "book" and "Book"). If you are searching for a specific bigram or trigram, enter it with spaces separating each part. Remember that if you deleted stopwords above, your bi- or trigram should not contain stopwords. Follow the below format exactly:

`token = 'unigram'`

OR

`token = 'bigram example'`

OR

`token = 'trigram example here'`

You need to put quotes around your token because you are entering it as a string.



In [ ]:
token = ''

Run the below cell to obtain the frequency count for your selected token. If your token as it is entered above does not appear in the corpus, the frequency count will be 0.

In [ ]:
# get frequency count
if set_length == 'unigram':
    print(token + ': ' + str(freq[token]))
if set_length == 'bigram':
    token1 = token.split()[0]
    token2 = token.split()[1]
    print(token + ': ' + str(freq[token1, token2]))
if set_length == 'trigram':
    token1 = token.split()[0]
    token2 = token.split()[1]
    token3 = token.split()[2]
    print(token + ': ' + str(freq[token1, token2, token3]))

**Relative Frequency**: The cells above gave you the raw counts of tokens in your selected document. However, raw counts can be misleading, as they do not account for the relative frequency of tokens within a document (i.e., how many times a given token occurs *relative* to all other tokens). Relative frequencies are obtained by dividing the raw frequency count for each token by the total number of tokens in a document. To do this, run the below cell. You must run the below cell to complete any other cells in this portion of the notebook. After running this cell, you can create a dataframe of relative frequencies, save that dataframe, plot the top results, and obtain the relative frequency of a given token.

In [ ]:
# obtain the total number of tokens in the document
norm = freq.N()

# make a copy of the freq object 
freq_dist = freq.copy()

# replace raw frequency counts with relative frequency values
for key in freq_dist.keys():
        freq_dist[key] = float(freq_dist[key]) / norm

Produce a dataframe of the relative frequency values for all tokens (uni-, bi-, or trigrams, whichever you selected above) in the document. The dataframe will be ordered by relaive frequency value, from the tokens with the highest relative frequency to those with the lowest. 

The values will be listed as decimals, and you should interpret them as percentage values. For example, if the top token listed is "town" with a relative frequency of .0244, then this means that the token "town" accounts for 2.44% of the total number of tokens in the document.

In [ ]:
df_relfrequencies = pd.Series(freq_dist, name='Frequency Distribution')

df_relfrequencies.index.name = 'Token'

df_relfrequencies = df_relfrequencies.reset_index()

df_relfrequencies = df_relfrequencies.sort_values('Frequency Distribution', ascending=False) 

**View the dataframe for these counts.** The below cell uses a [QGrid](https://github.com/quantopian/qgrid) widget to display count results in a dataframe. Click a column label to sort by that column. Click it again to reverse sort. Click the filter icon to the right of the column label to apply filters (for instance, reducing the table to only documents from specific sources). You can re-order the columns by dragging the column label.

In [ ]:
qgrid_widget = qgrid.show_grid(df_relfrequencies, grid_options=grid_options, show_toolbar=False)

qgrid_widget

**Save the dataframe to a csv file.** The cell below will save the version of the dataframe you see displayed in the cell above. To save the full version of the dataframe (disregarding any filtering, etc you have done in the qgrid dataframe), skip the next cell, uncomment the code in the cell below it, and run that cell. 

Either cell will create a csv file in this module directory called "PROJECT-NAME_LENGTH_relfreq.csv". You can download this csv for further processing using Excel, Google Sheets, etc.

In [ ]:
# create csv file name
csv_file_counts = project_dir + '_' + set_length + '_relfreq.csv'

# Save version of dataframe you see above to csv
changed_df = qgrid_widget.get_changed_df()

changed_df.to_csv(csv_file_counts, index_label = 'Index')

In [ ]:
# # create csv file name
# csv_file_counts = project_dir + '_' + set_length + '_relfreq.csv'

# # save the above dataframe to a csv file
# df_relfrequencies.to_csv(csv_file_counts, index_label = 'Index')

Plot the top n tokens (uni-, bi-, or trigrams, whatever you selected above) with their relative frequencies. It is not recommended that you plot a graph of *all* token frequencies in a document, even with stop words removed, as it will be hard to read. Instead, define the top number you want to see (i.e., the top 30 results), and plot those. The example provided below is 30. If you would like to change this, please follow the below format exactly:

`top_n = 40`

You do not need quotes around the number, as it is an integer, not a string.

In [ ]:
# define the number of top results you want to see in your plot
top_n = 30

In [ ]:
# plot frequency counts for your given document
%matplotlib inline

#plotting top 30 words
freq_dist.plot(top_n, title = 'Relative Frequencies')

Find the relative frequency of a specific token across all documents in your project. Check the `set_length` variable to make sure you have tokenized on the proper token length. I.e., if you are searching for a specific word, you should have tokenized on unigrams above when you ran the `frequency_dir` function. If you ran the function using the unigram setting, but would now like to search for a bigram or trigram, you must re-run the `frequeny_dir` function above (the first cell under "B) Run Code and Visualize Results").

In [ ]:
print(set_length)

Enter the token you want to search for. If you are searching for a unigram, just enter it below. Remember that the tokenization method is case-insensitive (i.e., it is not possible to differentiate between "book" and "Book"). If you are searching for a specific bigram or trigram, enter it with spaces separating each part. Remember that if you deleted stopwords above, your bi- or trigram should not contain stopwords. Follow the below format exactly:

`token = 'unigram'`

OR

`token = 'bigram example'`

OR

`token = 'trigram example here'`

You need to put quotes around your token because you are entering it as a string.

In [ ]:
token = ''

Run the below cell to obtain the relative frequency for your selected token. If your token as it is entered above does not appear in the corpus, the relative frequency will be 0.

In [ ]:
# get frequency count
if set_length == 'unigram':
    print(token + ': ' + str(freq_dist[token]))
if set_length == 'bigram':
    token1 = token.split()[0]
    token2 = token.split()[1]
    print(token + ': ' + str(freq_dist[token1, token2]))
if set_length == 'trigram':
    token1 = token.split()[0]
    token2 = token.split()[1]
    token3 = token.split()[2]
    print(token + ': ' + str(freq_dist[token1, token2, token3]))

## 4. Tf-idf Scores

Tf-idf, or term frequency - inverse document frequency, is a common way of measuring the importance of tokens both within a given document and across a corpus as a whole (i.e., across your project). We obtain a token's tf-idf score by multipling its relative frequency within a given document by the inverse of the number of documents that token appears in throughout the corpus. See https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089 for a more in-depth explanation of the math.

Generally speaking, tokens with higher tf-idf scores (those closer to 1) are more important to a given document or corpus. At the document level, "distinctive" is a rough synonym for "important;" tf-idf provides a way to discover the tokens that are most distinctive within each document in your project. At the corpus or project level, a higher average tf-idf score means that a token is more frequently a distinctive word for documents within your corpus, i.e., it is potentially an important token for understanding your corpus overall.

#### A) Configure Code

Make sure that you have run the cell at the very top of this notebook, under "SETTINGS", before proceeding. You must run all of the cells under this "Configure Code" section, even if you do not change the values.

The first step is to configure which field you would like to use as your input. You do this using the `content_field` variable. If your data includes full-text data in the `content` field, you can set `content_field = 'content'`. If you have data in your `bag_of_words` field, set `content_field = 'bag_of_words'`. If your data is in the `features` field, set `content_field = 'features'`. **Please note that if you use EITHER `bag_of_words` OR `features` as your text input field, you will only be able to count unigrams.**

You follow the below format exactly:

`content_field = 'content'`

You must include quotation marks around your word or phrase because you are storing it as a string in the `required_phrase` variable.

In [ ]:
## you may select 'content', 'features', or 'bag_of_words'

content_field = 'content'

Do you want to work with unigrams, bigrams or trigrams? Configure the `set_length` variable below according to what you would like to obtain tf-idf scores for. If you have a specifc token or tokens in mind that you want to explore, select the setting most appropriate for that token (i.e., if it is a unigram, use the unigram setting, etc.). The default setting is unigrams; to work with trigrams, comment out the bigram line, and uncomment the trigram line. **Only one line below should be uncommented when you run the cell.**

**Note:** Because this code does not strip hyphens (see below), hyphenated words like "first-generation" are considered to be one word.

In [ ]:
set_length = 'unigram'
# set_length = 'bigram'
# set_length = 'trigram'

Run the below cell to make sure you have set the `set_length` variable correctly. This cell will print the current value of the `set_length` variable. If you have set the wrong value, re-run the above cell with the correct setting.

In [ ]:
print(set_length)

The below cell strips the following punctuation from project documents. It will **NOT** strip hyphens, single or double, in order to account for hyphenated words and phrases such as "first-generation". Because this punctuation list is bespoke and not standardized (standardized options strip hyphens), some punctuation marks or other non-Unicode characters may make it through, although most should be stripped. You do not need to change anything about the below cell (unless you are interested in the frequency of punctuation marks, or @ signs, etc.), but you do need to run it.

In [ ]:
# define punctuation to strip
punctuations = "_______________________\'m\'n\'ve!()[]{};:\'\"\,<>./?@#$%^&*_~''``''"

Do you want to delete stop words before obtaining word frequencies? The default setting is to delete stop words, but if your data has already had the stop words you want removed, you should change the value of `set_stopwords` to `False`. It is also generally recommended to delete stop words from a document before obtaining bi- and/or trigram frequencies. This will result in "inexact" bi- and trigrams, however, as any stop words will be deleted *before* tokenization into bi- or trigrams. If you are interested in specific bi- or trigrams that contain stop words (such as "first generation", without a hyphen -- the stop word is "first"), you may want to create a custom stop word list. 

You can view the standard list of WE1S stopwords used for all of our processes in the project template folder (the relative path is `project_name/modules/topic_modeling/scripts/we1s_standard_stoplist.txt`). This is the stop word list that will be applied to your data. You can edit this file for your project as much as you would like. You can also use your own custom stop word list. If you would like to use a different stop word list, format that list so it looks like the WE1S stopword list (one word per line, plain text file), upload that list to your project and configure the `stopword_file` variable in the "Settings" cell to the custom stop word file's path.

If you would like to the stop word setting, change `True` to `False` below. It should look like this exactly:

`set_stopwords = False`

Unlike above, you do not need quotation marks around `False` because it is a Boolean, not a string.

In [ ]:
# set stop word setting 
# default is to delete stop words, which means setting set_stopwords to True.
# to keep stop words, set set_stopwords to False.
set_stopwords = True


#### B) Run Code and Visualize Results


In [ ]:
# obtain tf-idf scores
df_tfidf, vectors, feature_names, file_list, bad_jsons = tfidf_dir(json_dir, content_field, set_stopwords, 
                                                                   punctuations, set_length, stopword_file)


Run the below cell to see the number of json documents in your project that failed to load correctly (and so were not read or considered in creating count totals). If this number is large, this will significantly affect the results you can obtain using this notebook; notify Lindsay if this is the case.

In [ ]:
len(bad_jsons)

Check the number of total tokens in your project.

In [ ]:
len(df_tfidf.columns)

You could at this point print out a dataframe of all of the tf-idf scores across the whole project. But that would be a huge dataframe, and many of its cells would be empty because not every token across the project appears in every document. Instead, you can zero in on specific tokens you are interested in. You can use the below cells to find tf-idf scores for a specific token across all documents in your project. Check the `set_length` variable to make sure you have tokenized on the proper token length. I.e., if you are searching for a specific word, you should have tokenized on unigrams above when you ran the `tfidf_dir` function. If you ran the function using the unigram setting, but would now like to search for a bigram or trigram, you must re-run the `tfidf_dir` function above (the first cell under "B) Run Code and Visualize Results").

In [ ]:
print(set_length)

Enter the token you want to search for. If you are searching for a unigram, just enter it below. Remember that the tokenization method is case-insensitive (i.e., it is not possible to differentiate between "book" and "Book"). If you are searching for a specific bigram or trigram, enter it with spaces separating each part. Remember that if you deleted stopwords above, your bi- or trigram should not contain stopwords. Follow the below format exactly:

`token = 'unigram'`

OR

`token = 'bigram example'`

OR

`token = 'trigram example here'`

You need to put quotes around your token because you are entering it as a string.

In [ ]:
token = ''

Get tf-idf scores for that token. If your token as it is entered above does not appear in the corpus, this cell will print a statement notifying you.

In [ ]:
# get tf-idf scores
if set_length == 'unigram':
    try:
        df_tfidf_token = df_tfidf[[token]]
        df_tfidf_token = df_tfidf_token.loc[df_tfidf_token[token] > 0]
    except KeyError as err:
        print(token + ' not in corpus')
if set_length == 'bigram':
    token1 = token.split()[0]
    token2 = token.split()[1]
    token_tuple = (token1, token2)
    try: 
        df_tfidf_token = df_tfidf[[token_tuple]]
        df_tfidf_token = df_tfidf_token.loc[df_tfidf_token[token_tuple] > 0]
    except KeyError as err:
        print(token + ' not in corpus')
if set_length == 'trigram':
    token1 = token.split()[0]
    token2 = token.split()[1]
    token3 = token.split()[2]
    token_tuple = (token1, token2, token3)
    try:
        df_tfidf_token = df_tfidf[[token_tuple]]
        df_tfidf_token = df_tfidf_token.loc[df_tfidf_token[token_tuple] > 0]
    except KeyError as err:
        print(token + ' not in corpus')

**View a dataframe listing the filenames of the documents in which your selected token appears and its tf-idf score for each document in which it appears.** The below cell uses a [QGrid](https://github.com/quantopian/qgrid) widget to display count results in a dataframe. Click a column label to sort by that column. Click it again to reverse sort. Click the filter icon to the right of the column label to apply filters (for instance, reducing the table to only documents from specific sources). You can re-order the columns by dragging the column label.

In [ ]:
qgrid_widget = qgrid.show_grid(df_tfidf_token, grid_options=grid_options, show_toolbar=False)

qgrid_widget

We can also find the top_n tf-idf scores for any given document in your project. To do this, you must enter **the full file path** of the specific document you want to explore in more detail in the cell below. You must enter the full file path, which you can get from the dataframe above, or from the csv you downloaded in section 2. That file path should begin with something like `'/home/jovyan/write/projects/all-hands/'`, which is the path to most of our project folders. I have included that path below as a starting point. You need to enter the full path, following the below format exactly:

`fpath = 'full/filepath/here/it/should/end/in/.json'`

You need quotes around the file path because you are entering it as a string.

In [ ]:
# enter the full filepath for the document you want to explore

fpath = ''

Run the cell below to select the number of tf-idf scores for your selected document that you would like to see. The results will be sorted from highest to lowest, and the default is 50. If you would like to change this, please follow the below format exactly:

`top_n = 30`

You do not need to put quotes around the number because you are entering it as an integer.

In [ ]:
# set number of tokens to see
top_n = 50

In [ ]:
# get row index
row_id = file_list.index(fpath)

# get top tf-idf scores for the specific document
df_single = top_feats_in_doc(vectors, feature_names, row_id, top_n)

**View a dataframe listing the top tf-idf scores for your selected document.** The below cell uses a [QGrid](https://github.com/quantopian/qgrid) widget to display count results in a dataframe. Click a column label to sort by that column. Click it again to reverse sort. Click the filter icon to the right of the column label to apply filters (for instance, reducing the table to only documents from specific sources). You can re-order the columns by dragging the column label.

In [ ]:
qgrid_widget = qgrid.show_grid(df_single, grid_options=grid_options, show_toolbar=False)

qgrid_widget

To understand what words are important to your project overall, we can find the average tf-idf scores of all tokens in the project. By listing the tokens with the top n tf-idf scores across the project as a whole, we can understand what the most important words are across the whole project. Run the cell below to select the number of top results you would like to see. The default is 50. If you would like to change this, please follow the below format exactly:

`top_n = 30`

You do not need to put quotes around the number because you are entering it as an integer.

In [ ]:
# select number of top results to display
top_n = 50

Get average tf-idf scores for top_n tokens.

In [ ]:
# get top average tf-idf scores
df = top_mean_feats(vectors, feature_names, top_n, grp_ids=None)

**View a dataframe listing the average tf-dif scores for the the top_n tokens in your proejct.** The below cell uses a [QGrid](https://github.com/quantopian/qgrid) widget to display count results in a dataframe. Click a column label to sort by that column. Click it again to reverse sort. Click the filter icon to the right of the column label to apply filters (for instance, reducing the table to only documents from specific sources). You can re-order the columns by dragging the column label.

In [ ]:
qgrid_widget = qgrid.show_grid(df, grid_options=grid_options, show_toolbar=False)

qgrid_widget

## 5. Collocation Metrics for Bi- and Trigrams

**This section is probably only useful and meaningful for you if you are using full-text data as input (usually via the `content` field in your project json documents). See the module's README.md file for a fuller explanation of this issue. Therefore, the code in this section assumes that your data has a `content` field and it looks for full-text content there.**

It is also possible to understand the relationships between words in your project with a bit more precision. The metrics gathered under this section include five of the dozens of different ways of understanding collocation (the four gathered here are perhaps the most common, or at least are methods I've seen used among DH practitioners). Collocation is another way of discussing co-occurrence; in natural language processing, the term "collocation" usually refers to phrases of two or more words that commonly occur together in a given context. You can use this section to understand how common certain bi- and trigrams are in your project. Generally speaking, the more tokens you have in your project, and the larger your project data is, the more meaningful these metrics will be. 

**Since collocation always involves 2 or more words, this section only works for bi- and trigrams.**

For a brief introduction to the concept of collocation in natural language processing and to some of the metrics I use in this notebook, see https://medium.com/@nicharuch/collocations-identifying-phrases-that-act-like-individual-words-in-nlp-f58a93a2f84a. More in-depth explanations can be found in a NLP textbook chapter on collocations, here: https://nlp.stanford.edu/fsnlp/promo/colloc.pdf. If you would like to listen to an 11-minute lecture about collocation and natural language processing, see https://www.youtube.com/watch?v=Kf25kCKuOUI.

This notebook provides a limited variety of metrics. As linguist Gerlof Bouma writes, "One of the lessons taught by systematic evaluation of association measures against different gold standards is that there is not one association measure that is best in all situations. Rather, different target collocations may be found most effectively with different methods and measures. It is therefore useful to have access to a wide array of association measures coupled with an understanding of their behaviour if we want to do collocation extraction" (see https://pdfs.semanticscholar.org/1521/8d9c029cbb903ae7c729b2c644c24994c201.pdf). Understanding which metric is best for your question or context is a larger task than this notebook can tackle. If you want to go down this rabbit hole, I recommend starting with Bouma's widely cited article quoted in the previous sentence.

You must run the **Configure Code** cells below for the rest of the section to work. If you run the below cells the first time counting bigrams, you can re-run them counting trigrams if you change the settings appropriately. If you are re-running this section, you should start here and reconfigure the code before re-running anything.

#### A) Configure Code

Make sure that you have run the cell at the very top of this notebook, under "Settings", before proceeding. You must run all of the cells under this "Configure Code" section, even if you do not change the values.

In this section, your `content_field` variable must be set to `'content'`. The code in this section is only able to work with full-text data. You must run the below cell, but do not change its value.

You follow the below format exactly:

`content_field = 'content'`

You must include quotation marks around your word or phrase because you are storing it as a string in the `required_phrase` variable.

In [ ]:
## only 'content' will work correctly for this section

content_field = 'content'

Do you want to work with bigrams or trigrams? Since collocations always involve 2 or more words, this section of the notebook only works with bigrams and trigrams. Configure the `set_length` variable below according to what you would like to work with. The default is bigrams; to work with trigrams, comment out the bigram line, and uncomment the trigram line. **Only one line below should be uncommented when you run the cell.**

**Note:** Because this code does not strip hyphens (see below), hyphenated words like "first-generation" are considered to be one word.

In [ ]:
set_length = 'bigram'
# set_length = 'trigram'

Run the below cell to make sure you have set the `set_length` variable correctly. This cell will print the current value of the `set_length` variable. If you have set the wrong value, re-run the above cell with the correct setting.

In [ ]:
print(set_length)

The below cell strips the following punctuation from project documents. It will **NOT** strip hyphens, single or double, in order to account for hyphenated words and phrases such as "first-generation". Because this punctuation list is bespoke and not standardized (standardized options strip hyphens), some punctuation marks or other non-Unicode characters may make it through, although most should be stripped. You do not need to change anything about the below cell (unless you are interested in the frequency of punctuation marks, or @ signs, etc.), but you do need to run it.

In [ ]:
# define punctuation to strip
punctuations = "_______________________\'m\'n\'ve!()[]{};:\'\"\,<>./?@#$%^&*_~''``''"


Do you want to delete stop words before obtaining word frequencies? The default setting is to delete stop words, but if your data has already had the stop words you want removed, you should change the value of `set_stopwords` to `False`. It is also generally recommended to delete stop words from a document before obtaining bi- and/or trigram frequencies. This will result in "inexact" bi- and trigrams, however, as any stop words will be deleted *before* tokenization into bi- or trigrams. If you are interested in specific bi- or trigrams that contain stop words (such as "first generation", without a hyphen -- the stop word is "first"), you may want to create a custom stop word list. 

You can view the standard list of WE1S stopwords used for all of our processes in the project template folder (the relative path is `project_name/modules/topic_modeling/scripts/we1s_standard_stoplist.txt`). This is the stop word list that will be applied to your data. You can edit this file for your project as much as you would like. You can also use your own custom stop word list. If you would like to use a different stop word list, format that list so it looks like the WE1S stopword list (one word per line, plain text file), upload that list to your project and configure the `stopword_file` variable in the "Settings" cell to the custom stop word file's path.

If you would like to the stop word setting, change `True` to `False` below. It should look like this exactly:

`set_stopwords = False`

Unlike above, you do not need quotation marks around `False` because it is a Boolean, not a string.

In [ ]:
# set stop word setting 
# default is to delete stop words, which means setting set_stopwords to True.
# to keep stop words, set set_stopwords to False.
set_stopwords = True


Finally, the cell below obtains the frequency information you need for ALL metrics below. To obtain any collocation metrics in this section, you must run this cell. But you only need to run this cell once per each notebook session.

In [ ]:
# obtain frequency information you need
all_finders_freq, all_finders_list, freq, bad_jsons = frequency_dir(json_dir, content_field, set_stopwords, 
                                                                    punctuations, set_length, stopword_file)


#### B) Run for Different Association Metrics

**Likelihood ratio:** This metric is perhaps the easiest to understand. Likelihood ratios reflect the likelihood, within a given corpus (i.e., all documents in the project), of a specific bi- or trigram occurring (technically it tells us the likelihood that any two or three given words exist in a dependent relationship). The higher a likelihood score, the more strongly associated the words composing the bi- or trigram are with one another, roughly speaking. Likelihood ratios usually perform better than t-tests or chi-squared tests (see below) on sparse data (i.e., bigrams and trigrams), and so are often used in natural language processing. The code below is an implementation of Dunning's log likelihood test.

For more on likelihood ratios in natural language processing, see https://nlp.stanford.edu/fsnlp/promo/colloc.pdf (pages 161-164), and https://stackoverflow.com/questions/48715547/how-to-interpret-python-nltk-bigram-likelihood-ratios.

The cell below will run a likelihood ratio test for all tokens (bigrams or trigrams, depending on what you chose).

In [ ]:
metric = 'likelihood'

LikelihoodTable, all_scores = collocation_metric(set_length, all_finders_list, metric, freq_filter=None)

**View the dataframe for these scores.** The below cell uses a [QGrid](https://github.com/quantopian/qgrid) widget to display results in a dataframe, sorted from from highest to lowest. Click a column label to sort by that column. Click it again to reverse sort. Click the filter icon to the right of the column label to apply filters (for instance, reducing the table to only documents from specific sources). You can re-order the columns by dragging the column label.

In [ ]:
qgrid_widget = qgrid.show_grid(LikelihoodTable, grid_options=grid_options, show_toolbar=False)

qgrid_widget

**Save the dataframe to a csv file.** The cell below will save the version of the dataframe you see displayed in the cell above. To save the full version of the dataframe (disregarding any filtering, etc you have done in the qgrid dataframe), skip the next cell, uncomment the code in the cell below it, and run that cell. 

Either cell will create a csv file in this module directory called "likelihood_LENGTH_all.csv". You can download this csv for further processing using Excel, Google Sheets, etc.

In [ ]:
# create csv file name
csv_file = 'likelihood_' + set_length + '_all.csv'

# Save version of dataframe you see above to csv
changed_df = qgrid_widget.get_changed_df()

changed_df.to_csv(csv_file, index_label = 'Index')

In [ ]:
# csv_file = 'likelihood_' + set_length + '_all.csv'

# # save the above dataframe to a csv file
# LikelihoodTable.to_csv(csv_file, index_label = 'Index')

Run the below cell to see the number of json documents in your project that failed to load correctly (and so were not read or considered in creating count totals). If this number is large, this will significantly affect the results you can obtain using this notebook; notify Lindsay if this is the case.

In [ ]:
len(bad_jsons)

Would you like to check the likelihood ratios for a specific word within your project? This means you are checking to see what other words are highly associated with your chosen word across your project. You must enter only a single word below; it does not work if you enter a bigram or a trigram. Enter that word below following the below format exactly:

`token = 'example'`

You need to use quotes around the word because you are entering it as a string.

In [ ]:
token = ''

Run the below code to return any other word or words (depending on if you have computed bi- or trigrams) your provided word occurs with throughout your project, and the scores for each grouping. 

You may also choose to save this information to a csv file by changing the value of the following variable:

`save_csv = True`

Setting `save_csv` to `True` will create a csv file in this module's directory called  "likelihood_scores_LENGTH_TOKEN.csv" (it is set to `False` by default).

In [ ]:
save_csv = False
csv_file = 'likelihood_scores_' + set_length + '_' + token + '.csv'

token_scores = order_collocation_scores(all_scores, token, save_csv, csv_file)

token_scores

**Mutual Information score:** The MI score is a measure of the strength of association between any given token in a project and all of the project's tokens. An MI score measures how much more likely the words of a bi- or trigram are to co-occur within your project than they are to occur by themselves. A higher score indicates a higher strength of association. For more on this concept, see [this guide](https://wordbanks.harpercollins.co.uk/other_doc/statistics.html#:~:text=The%20Mutual%20Information%20score%20expresses,between%20words%20x%20and%20y) on the mutual information scores and t-tests (see below) in corpus linguistics.

The code below implements NLTK's version of mutual information. See NLTK documentation here: https://www.nltk.org/_modules/nltk/metrics/association.html

MI scores are sensitive to unique words, however, which can make results less meaningful because often unique words will occur much less frequently throughout a corpus. To account for this, we can set a frequency filter so that we only measure PMI scores for bi- or trigrams that occur a certain number of times. The default is to set a frequency filter. If you would like to NOT set a frequency filter, please change the value of `freq_filter` so that it looks like this: `freq_filter = None`.

If you would like to turn the frequency filter on, please provide a value for the frequency filter. This value determines the frequency cutoff. To change this, please follow the below format exactly (so, in this example, we are only measuring MI scores for bi- or trigrams that occur at least 5 times):

`freq_filter = 5`

You do not need quotes around the number because it is an integer.

In [ ]:
# set frequency cutoff you would like. 
freq_filter = 5

Run the below cell to generate a table ranking all of the bi- or trigrams in your project by MI score. You can change the settings above and re-run the below cell as many times as you like.

In [ ]:
metric = 'mi'

MITable, all_scores = collocation_metric(set_length, all_finders_list, metric, freq_filter=freq_filter)

**View the dataframe for these scores.** The below cell uses a [QGrid](https://github.com/quantopian/qgrid) widget to display results in a dataframe, sorted from from highest to lowest. Click a column label to sort by that column. Click it again to reverse sort. Click the filter icon to the right of the column label to apply filters (for instance, reducing the table to only documents from specific sources). You can re-order the columns by dragging the column label.

In [ ]:
qgrid_widget = qgrid.show_grid(MITable, grid_options=grid_options, show_toolbar=False)

qgrid_widget

**Save the dataframe to a csv file.** The cell below will save the version of the dataframe you see displayed in the cell above. To save the full version of the dataframe (disregarding any filtering, etc you have done in the qgrid dataframe), skip the next cell, uncomment the code in the cell below it, and run that cell. 

Either cell will create a csv file in this module directory called "mi_LENGTH_all.csv". You can download this csv for further processing using Excel, Google Sheets, etc.

In [ ]:
# create csv file name
csv_file = 'mi_' + set_length + '_all.csv'

# Save version of dataframe you see above to csv
changed_df = qgrid_widget.get_changed_df()

changed_df.to_csv(csv_file, index_label = 'Index')

In [ ]:
# csv_file = 'mi_' + set_length + '_all.csv'

# # save the above dataframe to a csv file
# MITable.to_csv(csv_file, index_label = 'Index')

Run the below cell to see the number of json documents in your project that failed to load correctly (and so were not read or considered in creating count totals). If this number is large, this will significantly affect the results you can obtain using this notebook; notify Lindsay if this is the case.

In [ ]:
len(bad_jsons)

Would you like to check the MI scores for a specific word within your project? This means you are checking to see what other words are highly associated with your chosen word across your project. You must enter only a single word below; it does not work if you enter a bigram or a trigram. Enter that word below following the below format exactly:

`token = 'example'`

You need to use quotes around the word because you are entering it as a string.

In [ ]:
token = ''

Run the below code to return any other word or words (depending on if you have computed bi- or trigrams) your provided word occurs with throughout your project, and the scores for each grouping. 

You may also choose to save this information to a csv file by changing the value of the following variable:

`save_csv = True`

Setting `save_csv` to `True` will create a csv file in this module's directory called  "mi_scores_LENGTH_TOKEN.csv".

In [ ]:
save_csv = True
csv_file = 'mi_scores_' + set_length + '_' + token + '.csv'

token_scores = order_collocation_scores(all_scores, token, save_csv, csv_file)

token_scores

**Pointwise Mutual Information (PMI) score:** PMI scores build on MI scores. Like MI scores, PMI scores measure the association between any given token in a project and all of the project's tokens. A PMI score measures how much more likely the words of a bi- or trigram are to co-occur within your project than they are to occur by themselves. A higher score indicates a higher strength of association. It differs from an MI score in that it refers to single comparisons, while MI scores are a measure of the average PMI scores over all comparisons. For more on this concept, see Gerlof Bouma's widely cited paper, "Normalized (Pointwise) Mutual Information in Collocation Extraction" (https://pdfs.semanticscholar.org/1521/8d9c029cbb903ae7c729b2c644c24994c201.pdf).

Like MI scores, PMI scores are sensitive to unique words, which can make results less meaningful because often unique words will occur much less frequently throughout a corpus. To account for this, we can set a frequency filter so that we only measure PMI scores for bi- or trigrams that occur a certain number of times. The default is to set a frequency filter. If you would like to NOT set a frequency filter, please change the value of `freq_filter` so that it looks like this: `freq_filter = None`.

If you would like to turn the frequency filter on, please provide a value for the frequency filter. This value determines the frequency cutoff. To change this, please follow the below format exactly (so, in this example, we are only measuring MI scores for bi- or trigrams that occur at least 5 times):

`freq_filter = 5`

You do not need quotes around the number because it is an integer.

In [ ]:
# set frequency cutoff you would like. 
freq_filter = 5

Run the below cell to generate a table ranking all of the bi- or trigrams in your project by PMI score. You can change the settings above and re-run the below cell as many times as you like.

In [ ]:
metric = 'pmi'

PMITable, all_scores = collocation_metric(set_length, all_finders_list, metric, freq_filter=freq_filter)

**View the dataframe for these scores.** The below cell uses a [QGrid](https://github.com/quantopian/qgrid) widget to display results in a dataframe, sorted from from highest to lowest. Click a column label to sort by that column. Click it again to reverse sort. Click the filter icon to the right of the column label to apply filters (for instance, reducing the table to only documents from specific sources). You can re-order the columns by dragging the column label.

In [ ]:
qgrid_widget = qgrid.show_grid(PMITable, grid_options=grid_options, show_toolbar=False)

qgrid_widget

**Save the dataframe to a csv file.** The cell below will save the version of the dataframe you see displayed in the cell above. To save the full version of the dataframe (disregarding any filtering, etc you have done in the qgrid dataframe), skip the next cell, uncomment the code in the cell below it, and run that cell. 

Either cell will create a csv file in this module directory called "pmi_LENGTH_all.csv". You can download this csv for further processing using Excel, Google Sheets, etc.

In [ ]:
# create csv file name
csv_file = 'pmi_' + set_length + '_all.csv'

# Save version of dataframe you see above to csv
changed_df = qgrid_widget.get_changed_df()

changed_df.to_csv(csv_file, index_label = 'Index')

In [ ]:
# csv_file = 'pmi_' + set_length + '_all.csv'

# # save the above dataframe to a csv file
# PMITable.to_csv(csv_file, index_label = 'Index')

Run the below cell to see the number of json documents in your project that failed to load correctly (and so were not read or considered in creating count totals). If this number is large, this will significantly affect the results you can obtain using this notebook; notify Lindsay if this is the case.

In [ ]:
len(bad_jsons)

Would you like to check the PMI scores for a specific word within your project? This means you are checking to see what other words are highly associated with your chosen word across your project. You must enter only a single word below; it does not work if you enter a bigram or a trigram. Enter that word below following the below format exactly:

`token = 'example'`

You need to use quotes around the word because you are entering it as a string.

In [ ]:
token = ''

Run the below code to return any other word or words (depending on if you have computed bi- or trigrams) your provided word occurs with throughout your project, and the scores for each grouping. 

You may also choose to save this information to a csv file by changing the value of the following variable:

`save_csv = True`

Setting `save_csv` to `True` will create a csv file in this module's directory called  "pmi_scores_LENGTH_TOKEN.csv".

In [ ]:
save_csv = True
csv_file = 'pmi_scores_' + set_length + '_' + token + '.csv'

token_scores = order_collocation_scores(all_scores, token, save_csv, csv_file)

token_scores

**Student's t-test**: Perhaps one of the most widely used methods of testing a hypothesis. This implementation of the t-test assumes that the words in any bi- or trigram are independent, and it measures how likely the words are to appear together in your project. Like the PMI score, a higher t-test score indicates a higher likelihood that the words in the bi- or trigram occur together in your project than that they occur separately. However, t-tests and chi-square tests (see below) have been shown to not perform as well with sparse data like bigrams and trigrams.

You can find a good general discussion of what a t-test is here: https://towardsdatascience.com/inferential-statistics-series-t-test-using-numpy-2718f8f9bf2f.

Run the below cell to generate a table ranking all of the bi- or trigrams in your project by t-test score. You can change the settings above and re-run the below cell as many times as you like.

In [ ]:
metric = 't-test'

TTable, all_scores = collocation_metric(set_length, all_finders_list, metric)

**View the dataframe for these scores.** The below cell uses a [QGrid](https://github.com/quantopian/qgrid) widget to display results in a dataframe, sorted from from highest to lowest. Click a column label to sort by that column. Click it again to reverse sort. Click the filter icon to the right of the column label to apply filters (for instance, reducing the table to only documents from specific sources). You can re-order the columns by dragging the column label.

In [ ]:
qgrid_widget = qgrid.show_grid(TTable, grid_options=grid_options, show_toolbar=False)

qgrid_widget

**Save the dataframe to a csv file.** The cell below will save the version of the dataframe you see displayed in the cell above. To save the full version of the dataframe (disregarding any filtering, etc you have done in the qgrid dataframe), skip the next cell, uncomment the code in the cell below it, and run that cell. 

Either cell will create a csv file in this module directory called "ttest_LENGTH_all.csv". You can download this csv for further processing using Excel, Google Sheets, etc.

In [ ]:
# create csv file name
csv_file = 'ttest_' + set_length + '_all.csv'

# Save version of dataframe you see above to csv
changed_df = qgrid_widget.get_changed_df()

changed_df.to_csv(csv_file, index_label = 'Index')

In [ ]:
# csv_file = 'ttest_' + set_length + '_all.csv'

# # save the above dataframe to a csv file
# TTable.to_csv(csv_file, index_label = 'Index')

Run the below cell to see the number of json documents in your project that failed to load correctly (and so were not read or considered in creating count totals). If this number is large, this will significantly affect the results you can obtain using this notebook; notify Lindsay if this is the case.

In [ ]:
len(bad_jsons)

Would you like to check the student's t-test scores for a specific word within your project? This means you are checking to see what other words are highly associated with your chosen word across your project. You must enter only a single word below; it does not work if you enter a bigram or a trigram. Enter that word below following the below format exactly:

`token = 'example'`

You need to use quotes around the word because you are entering it as a string.

In [ ]:
token = ''

Run the below code to return any other word or words (depending on if you have computed bi- or trigrams) your provided word occurs with throughout your project, and the scores for each grouping. 

You may also choose to save this information to a csv file by changing the value of the following variable:

`save_csv = True`

Setting `save_csv` to `True` will create a csv file in this module's directory called  "ttest_scores_LENGTH_TOKEN.csv".

In [ ]:
save_csv = True
csv_file = 'ttest_scores_' + set_length + '_' + token + '.csv'

token_scores = order_collocation_scores(all_scores, token, save_csv, csv_file)

token_scores

**Chi-Square Test:** Another test of statistical significance. Like a t-test, a chi-squared test assumes that the words in any bi- or trigram are independent. But unlike a t-test, a chi-squared test does not assume a normal distribution. The code below uses an implementation of Pearson's chi-square test of association. As with PMI and t-test scores, a higher chi-squared test score indicates a greater degree of likelihood, i.e., a higher likelihood that the words in a given bi- or trigram occur together in your project.

Decent explanations of the chi-squared test can be found at the following links: https://www.analyticsvidhya.com/blog/2019/11/what-is-chi-square-test-how-it-works/, https://towardsdatascience.com/chi-square-test-for-feature-selection-in-machine-learning-206b1f0b8223. 

Run the below cell to generate a table ranking all of the bi- or trigrams in your project by t-test score. You can change the settings above and re-run the below cell as many times as you like.

In [ ]:
metric = 'chi-square'

ChiSqTable, all_scores = collocation_metric(set_length, all_finders_list, metric)

**View the dataframe for these scores.** The below cell uses a [QGrid](https://github.com/quantopian/qgrid) widget to display results in a dataframe, sorted from from highest to lowest. Click a column label to sort by that column. Click it again to reverse sort. Click the filter icon to the right of the column label to apply filters (for instance, reducing the table to only documents from specific sources). You can re-order the columns by dragging the column label.

In [ ]:
qgrid_widget = qgrid.show_grid(ChiSqTable, grid_options=grid_options, show_toolbar=False)

qgrid_widget

**Save the dataframe to a csv file.** The cell below will save the version of the dataframe you see displayed in the cell above. To save the full version of the dataframe (disregarding any filtering, etc you have done in the qgrid dataframe), skip the next cell, uncomment the code in the cell below it, and run that cell. 

Either cell will create a csv file in this module directory called "chisquare_LENGTH_all.csv". You can download this csv for further processing using Excel, Google Sheets, etc.

In [ ]:
# create csv file name
csv_file = 'chisquare_' + set_length + '_all.csv'

# Save version of dataframe you see above to csv
changed_df = qgrid_widget.get_changed_df()

changed_df.to_csv(csv_file, index_label = 'Index')

In [ ]:
# csv_file = 'chisquare_' + set_length + '_all.csv'

# # save the above dataframe to a csv file
# ChiSqTable.to_csv(csv_file, index_label = 'Index')

Run the below cell to see the number of json documents in your project that failed to load correctly (and so were not read or considered in creating count totals). If this number is large, this will significantly affect the results you can obtain using this notebook; notify Lindsay if this is the case.

In [ ]:
len(bad_jsons)

Would you like to check the scores for a specific word within your project? This means you are checking to see what other words are highly associated with your chosen word across your project. You must enter only a single word below; it does not work if you enter a bigram or a trigram. Enter that word below following the below format exactly:

`token = 'example'`

You need to use quotes around the word because you are entering it as a string.

In [ ]:
token = ''

Run the below code to return any other word or words (depending on if you have computed bi- or trigrams) your provided word occurs with throughout your project, and the scores for each grouping. 

You may also choose to save this information to a csv file by changing the value of the following variable:

`save_csv = True`

Setting `save_csv` to `True` will create a csv file in this module's directory called  "chisquare_scores_LENGTH_TOKEN.csv".

In [ ]:
save_csv = True
csv_file = 'chisquare_scores_' + set_length + '_' + token + '.csv'

token_scores = order_collocation_scores(all_scores, token, save_csv, csv_file)

token_scores

**Compare all scores to one another**: If you ran all of the collocation metrics in this section and would like to produce a dataframe that lists all of the top_n tokens from each metric together (and also includes raw frequency count) for easier comparison, run the final cells in this notebook. Doing this also more easily shows how different results from these different association metrics can be.

Run the below cell to select the number of bi- or trigrams to display. This number represents the top n results from each metric, i.e., if `top_n = 20`, you are selecting to display the top 20 results from each metric in your comparison table. If you want to change this number, follow this format exactly:

`top_n = 30`

You do not need quotes around the number because it is an integer.

In [ ]:
top_n = 20

Create your comparison table. You must have created dataframes listing the results of each collocation metric discussed in this section of the notebook for this to work (i.e., you must have completed all subsections of section 6 of this notebook prior to running to this cell).

In [ ]:
likelihood_top = LikelihoodTable[:top_n].token.values
mi_top = MITable[:topn].token.values
pmi_top = PMITable[:top_n].token.values
ttest_top = TTable[:top_n].token.values
chisq_top = ChiSqTable[:top_n].token.values

FreqTable = pd.DataFrame(list(all_finders_freq.items()), columns=['token','freq']).sort_values(by='freq', ascending=False)
freq_top = FreqTable[:top_n].token.values

Compare = pd.DataFrame([freq_top, likelihood_top, mi_top, pmi_top, ttest_top, chisq_top]).T

Compare.columns = ['Frequency Count','Likelihood Ratio', 'MI', 'PMI', 'T-test', 'Chi-Sq Test']

**View the dataframe for these scores.** The below cell uses a [QGrid](https://github.com/quantopian/qgrid) widget to display combined results in a dataframe. Click a column label to sort by that column. Click it again to reverse sort. Click the filter icon to the right of the column label to apply filters (for instance, reducing the table to only documents from specific sources). You can re-order the columns by dragging the column label.

In [ ]:
qgrid_widget = qgrid.show_grid(Compare, grid_options=grid_options, show_toolbar=False)

qgrid_widget

**Save the dataframe to a csv file.** The cell below will save the version of the dataframe you see displayed in the cell above. To save the full version of the dataframe (disregarding any filtering, etc you have done in the qgrid dataframe), skip the next cell, uncomment the code in the cell below it, and run that cell. 

Either cell will create a csv file in this module directory called "collocation_metrics_compare_top_TOP_N.csv". You can download this csv for further processing using Excel, Google sheets, etc.

In [ ]:
# create csv file name
csv_compare_file = 'collocation_metrics_compare_top_' + top_n + '.csv'

# Save version of dataframe you see above to csv
changed_df = qgrid_widget.get_changed_df()

changed_df.to_csv(csv_compare_file, index_label = 'Index')

In [ ]:
# csv_compare_file = 'collocation_metrics_compare_top_' + top_n + '.csv'

# # save the above dataframe to a csv file
# Compare.to_csv(csv_compare_file, index_label = 'Index')